In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile, sys, os, pickle
sys.path.insert(0, os.path.abspath('../..'))
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D
from keras import regularizers
from keras import backend as K
import numpy as np
import tensorflow as tf
import scipy
print ("Using TF:", tf.__version__)
print ("Using Keras:", keras.__version__)

# Import DASP
from dasp import DASP, ImagePlayerIterator

#Import DeepLift
# Installation instructions: https://github.com/kundajelab/deeplift
import deeplift
from deeplift.layers import NonlinearMxtsMode
from deeplift.conversion import kerasapi_conversion as kc
from deeplift.util import compile_func

# Import Deep Explain (for Grad * Input, Integrated Gradients and Occlusion implementations)
# Installation instructions: https://github.com/marcoancona/DeepExplain
from deepexplain.tensorflow import DeepExplain

Using TensorFlow backend.


Using TF: 1.11.0
Using Keras: 2.2.4


In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image


# create the base pre-trained model
model = InceptionV3(weights='imagenet', include_top=True)

In [8]:
img_path = 'images/0c7ac4a8c9dfa802.png'
img = image.load_img(img_path, target_size=(299, 299))
xs = image.img_to_array(img)
xs = np.expand_dims(xs, axis=0)
#xs = preprocess_input(xs)

ys = model.predict(xs)
ys = np.array(ys == np.max(ys, 1), dtype='int32')

a_intgrad = np.zeros_like(xs)
a_res = np.zeros_like(xs)
a_rc = np.zeros_like(xs)
a_occlusion = np.zeros_like(xs)
a_dasp = np.zeros_like(xs)
a_sampling = np.zeros_like(xs)

In [10]:
# Clone the model to remove last softmax activation

reduced_model = Model(model.inputs, model.layers[-1].output)
reduced_model.layers[-1].activation = 'linear'
# Init DASP
dasp = DASP(reduced_model, player_generator=ImagePlayerIterator(xs, window_shape=(1,1,3)))
# Get model description (optional)
dasp.model_summary()

nplayers 89401


RuntimeError: Layer <keras.layers.normalization.BatchNormalization object at 0x7f7593599438> not supported